In [ ]:
import sys
import os

import h5py

import numpy as np

from astropy.io import fits

import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

from mrileyowens.stats import weighted_quantile

In [ ]:
def compare():

    '''
    Compare absolute UV magnitude measurements of the photometric catalog of F775W dropouts from Endsley et al. (2024) by Endsley et al. (2024) and my own
    measurements based on my BEAGLE fits to the catalog
    '''

    # Set common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'
    results = f'{home}/results'

    # Get the photometric catalog of F775W dropouts from Endsley et al. (2024)
    catalog = fits.open(f'{data}/JADES_f775w_dropouts_Endsley2024.fits')

    # Open my own measurements of the absolute UV magnitude from the BEAGLE fits I made to the Endsley et al. (2024) photometric catalog of F775W dropouts
    with h5py.File(f'{results}/m_uv/endsley2024_f775w_dropouts_beagle_csfh_fits_m_uv.h5', 'r') as f:

        # Get the IDs of the objects
        ids = f.keys()

        # Make a figure to plot the two sets of measurements against each other
        fig, ax = plt.subplots()

        # For each object ID
        for i, id in enumerate(ids):

            # Calculate the object ID's matching index in the Endsley et al. (2024) catalog
            idx_endsley2024 = np.where(catalog[1].data['ID'] == id)

            # Get the median, 16th, and 84th percentiles of the absolute UV magnitudes of Endsley et al. (2024)
            m_uv_endsley2024, m_uv_endsley2024_lower, m_uv_endsley2024_upper = catalog[1].data['Muv'][idx_endsley2024][0], catalog[1].data['Muv_l16'][idx_endsley2024][0], catalog[1].data['Muv_u84'][idx_endsley2024][0]

            # Get the posteriors and corresponding probabilities of my own absolute UV magnitude measurements of the Endsley et al. (2024) F775W dropout catalog
            m_uv_me_posterior, probabilities = f[id]['m_uv'][:], f[id]['probabilities'][:]

            # Calculate the median, 16th, and 84th percentiles of the posteriors of my own absolute UV magnitude measurements
            m_uv_me, m_uv_me_lower, m_uv_me_upper = weighted_quantile(m_uv_me_posterior, probabilities, 0.5), weighted_quantile(m_uv_me_posterior, probabilities, 0.16), weighted_quantile(m_uv_me_posterior, probabilities, 0.84)

            # Plot the two sets of measurements against each other
            ax.errorbar(m_uv_me, m_uv_endsley2024, xerr=[[abs(m_uv_me_lower - m_uv_me)],[abs(m_uv_me_upper - m_uv_me)]], yerr=[[abs(m_uv_endsley2024_lower - m_uv_endsley2024)], [abs(m_uv_endsley2024_upper - m_uv_endsley2024)]], 
                color='black')
            
        # Create a one-to-one line on the figure
        ax.axline((-20,-20), slope=1, color='red', linestyle='dashed')

        # Label the axes
        ax.set_xlabel(r'$M_\text{UV}$ (me)')
        ax.set_ylabel(r'$M_\text{UV}$ (Endsley et al. (2024))')

        # Set the title of the figure
        ax.set_title(r'Comparing $M_\text{UV}$ measurements of the F775W dropouts from Endsley et al. (2024)')

        # Save the figure
        fig.savefig(f'{figs}/m_uv_measurements_comparison.png', dpi=200, bbox_inches='tight')

def compare_diff():

    '''
    '''

    # Set common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'
    results = f'{home}/results'

    # Get the photometric catalog of F775W dropouts from Endsley et al. (2024)
    catalog = fits.open(f'{data}/JADES_f775w_dropouts_Endsley2024.fits')

    # Open my own measurements of the absolute UV magnitude from the BEAGLE fits I made to the Endsley et al. (2024) photometric catalog of F775W dropouts
    with h5py.File(f'{results}/m_uv/endsley2024_f775w_dropouts_beagle_csfh_fits_m_uv.h5', 'r') as f:

        # Get the IDs of the objects
        ids = np.array([key for key in f.keys()], dtype=str)

        # Make a figure to plot the two sets of measurements against each other
        #fig, ax = plt.subplots()

        diffs = []

        # For each object ID
        for i, id in enumerate(ids):

            # Calculate the object ID's matching index in the Endsley et al. (2024) catalog
            idx_endsley2024 = np.where(catalog[1].data['ID'] == id)

            # Get the median, 16th, and 84th percentiles of the absolute UV magnitudes of Endsley et al. (2024)
            m_uv_endsley2024, _, _ = catalog[1].data['Muv'][idx_endsley2024][0], catalog[1].data['Muv_l16'][idx_endsley2024][0], catalog[1].data['Muv_u84'][idx_endsley2024][0]

            # Get the posteriors and corresponding probabilities of my own absolute UV magnitude measurements of the Endsley et al. (2024) F775W dropout catalog
            m_uv_me_posterior, probabilities = f[id]['m_uv'][:], f[id]['probabilities'][:]

            # Calculate the median, 16th, and 84th percentiles of the posteriors of my own absolute UV magnitude measurements
            m_uv_me, _, _ = weighted_quantile(m_uv_me_posterior, probabilities, 0.5), weighted_quantile(m_uv_me_posterior, probabilities, 0.16), weighted_quantile(m_uv_me_posterior, probabilities, 0.84)

            diffs.append(abs(m_uv_endsley2024 - m_uv_me))

            # Plot the two sets of measurements against each other
            #ax.errorbar(m_uv_me, m_uv_endsley2024, xerr=[[abs(m_uv_me_lower - m_uv_me)],[abs(m_uv_me_upper - m_uv_me)]], yerr=[[abs(m_uv_endsley2024_lower - m_uv_endsley2024)], [abs(m_uv_endsley2024_upper - m_uv_endsley2024)]], 
            #    color='black')
            
        diffs = np.array(diffs, dtype=np.float64)
            
        ids_diffs = ids[diffs >= 0.2]

        print('Number of objects with a >= 0.2 mag difference between the two sets of M_UV measurements:', len(ids_diffs), '\n')
        print('Object IDs with a >= 0.2 mag difference between the two sets of M_UV measurements:', ids_diffs)
            
        # Create a one-to-one line on the figure
        #ax.axline((-20,-20), slope=1, color='red', linestyle='dashed')

        ## Label the axes
        #ax.set_xlabel(r'$M_\text{UV}$ (me)')
        #ax.set_ylabel(r'$M_\text{UV}$ (Endsley et al. (2024))')

        ## Set the title of the figure
        #ax.set_title(r'Comparing $M_\text{UV}$ measurements of the F775W dropouts from Endsley et al. (2024)')

        ## Save the figure
        #fig.savefig(f'{figs}/m_uv_measurements_comparison.png', dpi=200, bbox_inches='tight')

def bins():

    # Set common directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'

    # ------------------------------------------------------------------------------------------------------------
    # Determine the object IDs in each absolute UV magnitude bin according to Endsley et al. (2024)'s measurements
    # ------------------------------------------------------------------------------------------------------------

    # Get the photometric catalog of F775W dropouts from Endsley et al. (2024)
    catalog = fits.open(f'{data}/JADES_f775w_dropouts_Endsley2024.fits')

    # Get the object IDs from the photometric catalog of F775W dropouts from Endsley et al. (2024)
    ids_endsley2024 = catalog[1].data['ID']

    # Get the median, 16th, and 84th percentiles of the absolute UV magnitudes of Endsley et al. (2024)
    m_uv_endsley2024 = catalog[1].data['Muv']

    # Get the groups of IDs in each absolute UV magnitude bin
    ids_endsley2024_bright = ids_endsley2024[m_uv_endsley2024 <= -19.5]
    ids_endsley2024_faint = ids_endsley2024[(m_uv_endsley2024 > -19.5) & (m_uv_endsley2024 <= -18)]
    ids_endsley2024_vfaint = ids_endsley2024[m_uv_endsley2024 > -18]

    # ------------------------------------------------------------------------------------------------------------
    # Determine the object IDs in each absolute UV magnitude bin according to my measurements
    # ------------------------------------------------------------------------------------------------------------

    # Open my own measurements of the absolute UV magnitude from the BEAGLE fits I made to the Endsley et al. (2024) photometric catalog of F775W dropouts
    with h5py.File(f'{results}/m_uv/endsley2024_f775w_dropouts_beagle_csfh_fits_m_uv.h5', 'r') as f:

        # Get the IDs of the objects
        ids = np.array([key for key in f.keys()], dtype=str)

        m_uvs_me = []

        # For each object ID
        for i, id in enumerate(ids):

            # Get the posteriors and corresponding probabilities of my own absolute UV magnitude measurements of the Endsley et al. (2024) F775W dropout catalog
            m_uv_me_posterior, probabilities = f[id]['m_uv'][:], f[id]['probabilities'][:]

            # Calculate the median, 16th, and 84th percentiles of the posteriors of my own absolute UV magnitude measurements
            m_uv_me = weighted_quantile(m_uv_me_posterior, probabilities, 0.5)

            m_uvs_me.append(m_uv_me)

        m_uvs_me = np.array(m_uvs_me, dtype=np.float64)

        ids_me_bright = ids[m_uvs_me <= -19.5]
        ids_me_faint = ids[(m_uvs_me > -19.5) & (m_uvs_me <= -18)]
        ids_me_vfaint = ids[m_uvs_me > -18]

        print('The number of objects in Endsley et al. (2024)\'s bright (M_UV <= -19.5) selection: ', len(ids_endsley2024_bright))
        print('The number of objects in Endsley et al. (2024)\'s faint (-19.5 < M_UV <= -18) selection: ', len(ids_endsley2024_faint))
        print('The number of objects in Endsley et al. (2024)\'s very faint (M_UV > -18) selection: ', len(ids_endsley2024_vfaint), '\n')

        print('The number of objects in my bright (M_UV <= -19.5) selection: ', len(ids_me_bright))
        print('The number of objects in my faint (-19.5 < M_UV <= -18) selection: ', len(ids_me_faint))
        print('The number of objects in my very faint (M_UV > -18) selection: ', len(ids_me_vfaint), '\n')

        print('The number of objects in both bright (M_UV <= -19.5) selections: ', len(np.intersect1d(ids_endsley2024_bright, ids_me_bright)))
        print('The number of objects in both faint (-19.5 < M_UV <= -18) selections: ', len(np.intersect1d(ids_endsley2024_faint, ids_me_faint)))
        print('The number of objects in both very faint (M_UV > -18) selections: ', len(np.intersect1d(ids_endsley2024_vfaint, ids_me_vfaint)))

In [ ]:
compare()

In [73]:
compare_diff()

Number of objects with a >= 0.2 mag difference between the two sets of M_UV measurements: 17 

Object IDs with a >= 0.2 mag difference between the two sets of M_UV measurements: ['106549_GOODSN' '110913_GOODSS' '12565_GOODSN' '14870_GOODSN'
 '16781_GOODSN' '28474_GOODSN' '31101_GOODSN' '43154_GOODSN'
 '50244_GOODSN' '56603_GOODSN' '63938_GOODSN' '66959_GOODSS'
 '86604_GOODSS' '87760_GOODSS' '90985_GOODSS' '94758_GOODSS'
 '96735_GOODSN']


In [62]:
bins()

The number of objects in Endsley et al. (2024)'s bright (M_UV <= -19.5) selection:  65
The number of objects in Endsley et al. (2024)'s faint (-19.5 < M_UV <= -18) selection:  138
The number of objects in Endsley et al. (2024)'s very faint (M_UV > -18) selection:  75 

The number of objects in my bright (M_UV <= -19.5) selection:  57
The number of objects in my faint (-19.5 < M_UV <= -18) selection:  135
The number of objects in my very faint (M_UV > -18) selection:  82 

The number of objects in both bright (M_UV <= -19.5) selections:  57
The number of objects in both faint (-19.5 < M_UV <= -18) selections:  127
The number of objects in both very faint (M_UV > -18) selections:  72
